# [1070. Product Sales Analysis III](https://leetcode.com/problems/product-sales-analysis-iii/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Sales

<pre>+-------------+-------+
| Column Name | Type  |
+-------------+-------+
| sale_id     | int   |
| product_id  | int   |
| year        | int   |
| quantity    | int   |
| price       | int   |
+-------------+-------+</pre>
(sale_id, year) is the primary key (combination of columns with unique values) of this table.
product_id is a foreign key (reference column) to Product table.
Each row of this table shows a sale on the product product_id in a certain year.
Note that the price is per unit.
 

Table: Product

<pre>+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| product_id   | int     |
| product_name | varchar |
+--------------+---------+</pre>
product_id is the primary key (column with unique values) of this table.
Each row of this table indicates the product name of each product.
 

Write a solution to select the product id, year, quantity, and price for the first year of every product sold.

Return the resulting table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Sales table:
<pre>+---------+------------+------+----------+-------+
| sale_id | product_id | year | quantity | price |
+---------+------------+------+----------+-------+ 
| 1       | 100        | 2008 | 10       | 5000  |
| 2       | 100        | 2009 | 12       | 5000  |
| 7       | 200        | 2011 | 15       | 9000  |
+---------+------------+------+----------+-------+</pre>
Product table:
<pre>+------------+--------------+
| product_id | product_name |
+------------+--------------+
| 100        | Nokia        |
| 200        | Apple        |
| 300        | Samsung      |
+------------+--------------+</pre>
Output: 
<pre>+------------+------------+----------+-------+
| product_id | first_year | quantity | price |
+------------+------------+----------+-------+ 
| 100        | 2008       | 10       | 5000  |
| 200        | 2011       | 15       | 9000  |
+------------+------------+----------+-------+</pre>

In [1]:
# pandas schema

import pandas as pd

data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype(
    {'sale_id': 'Int64', 'product_id': 'Int64', 'year': 'Int64', 'quantity': 'Int64', 'price': 'Int64'})
data = [[100, 'Nokia'], [200, 'Apple'], [300, 'Samsung']]
product = pd.DataFrame(data, columns=['product_id', 'product_name']).astype(
    {'product_id': 'Int64', 'product_name': 'object'})

In [2]:
#converting to spark dataframe
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

sales_df = spark.createDataFrame(sales)
sales_df.show()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/11 16:46:42 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/11 16:46:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/11 16:46:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------+----------+----+--------+-----+
|sale_id|product_id|year|quantity|price|
+-------+----------+----+--------+-----+
|      1|       100|2008|      10| 5000|
|      2|       100|2009|      12| 5000|
|      7|       200|2011|      15| 9000|
+-------+----------+----+--------+-----+


In [3]:
product_df = spark.createDataFrame(product)
product_df.show()

+----------+------------+
|product_id|product_name|
+----------+------------+
|       100|       Nokia|
|       200|       Apple|
|       300|     Samsung|
+----------+------------+


In [4]:
from pyspark.sql import Window

# solving in spark dataframe

sales_df \
    .withColumn('dn', F.dense_rank().over(Window.partitionBy('product_id').orderBy('year')))\
    .filter(F.col('dn')==1)\
    .select('product_id', F.col('year').alias('first_year'),'quantity','price')\
    .show()

+----------+----------+--------+-----+
|product_id|first_year|quantity|price|
+----------+----------+--------+-----+
|       100|      2008|      10| 5000|
|       200|      2011|      15| 9000|
+----------+----------+--------+-----+


In [5]:
# solving spark sql

sales_df.createOrReplaceTempView('sales')
product_df.createOrReplaceTempView('product')

spark.sql('''
select  
    product_id,
    year as first_year,
    quantity,
    price
from
(select 
    sales.*,
    dense_rank() over (partition by sales.product_id order by year) as dn
from sales
inner join product
on sales.product_id = product.product_id) t
where t.dn =1;
''').show()

+----------+----------+--------+-----+
|product_id|first_year|quantity|price|
+----------+----------+--------+-----+
|       100|      2008|      10| 5000|
|       200|      2011|      15| 9000|
+----------+----------+--------+-----+


In [6]:
spark.stop()